# TESTING and SUBMITTING
By appending and executing this notebook at the end of training, we are able to test our model and evaluate its accuracy.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from PIL import Image
from os.path import join
from os import listdir, getcwd

#### Perform RLE encoding given the image

In [ ]:
# img: numpy array, 1 - foreground, 0 - background
# Returns run length as string formatted 

def rle_encode(img):
   
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
   
    return ' '.join(str(x) for x in runs)

#### Get image crop from the name

In [1]:
def get_crop(img_name):
    crop = "Haricot" if "haricot" in img_name else "Mais"
    return crop

#### Get image infos from the name, such as team, crop, and shape

In [2]:
def get_data(img_name):
    
    if "Bipbip" in img_name:
        return "Bipbip", get_crop(img_name), [1536,2048]
    
    elif "Pead" in img_name:   
        return "Pead", get_crop(img_name), [2464,3280]
        
    elif "Roseau" in img_name:   
        return "Roseau", get_crop(img_name), [819,1228]
        
    else:  
        return "Weedelec", get_crop(img_name), [3456,5184]

#### example:

In [3]:
get_data('Roseau_haricot_0010_false.png')

('Roseau', 'Haricot', [819, 1228])

#### Compute prediction on test images, and create dictionary out of it

In [ ]:
import json

submission_dict = {}
test_dir = os.path.join('/content/drive/My Drive','test_Set')

for image in os.listdir(test_dir):

    # Take image name by removing the extension
    img_name = image[:-4]
    image = os.path.join(test_dir,image)
    img = Image.open(image) 
    img = img.resize([1024,1024], Image.ANTIALIAS)
  
    team, crop, shape = get_data(img_name)

    expanded_im = tf.expand_dims(np.array(img), axis=0)

    # PREDICT
    prediction = model.predict(expanded_im)
    
    mask_arr = tf.image.resize(prediction, shape, method='nearest')
    mask_arr = tf.argmax(mask_arr, -1)[0, ...]
    mask_arr = np.array(mask_arr)

    submission_dict[img_name] = {}
    submission_dict[img_name]['shape'] = mask_arr.shape
    submission_dict[img_name]['team'] = team
    submission_dict[img_name]['crop'] = crop
    submission_dict[img_name]['segmentation'] = {}

    rle_encoded_crop = rle_encode(mask_arr == 1)
    rle_encoded_weed = rle_encode(mask_arr == 2)
    submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
    submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed


#### Finally, save the results into the json file for the submission

In [ ]:
with open(join('/content/drive/My Drive','submission.json'), 'w') as f:
    json.dump(submission_dict, f)